In [ ]:
#@title PDB + SMILES input

PDB_id = '' #@param {type:"string"}
SMILES_or_pubchem_id = '' #@param {type:"string"}

#@markdown Download a tar file containing all results?
download_results = True #@param {type:"boolean"}

In [1]:
#@title Run GNINA 1.0
#https://doi.org/10.26434/chemrxiv.13578140.v1
#https://github.com/gnina/gnina
#https://colab.research.google.com/drive/1QYo5QLUE80N_G28PlpYs6OKGddhhd931?usp=sharing


# Configure colab to use a GPU (Runtme->Change runtime type->Hardware Accelerator=GPU)
!nvidia-smi

# Download the linux binary and make it executable.
!wget https://github.com/gnina/gnina/releases/download/v1.0.3/gnina

!chmod +x gnina
!./gnina --version

# Get a pdb and extract the receptor and ligand.
!wget http://files.rcsb.org/download/3ERK.pdb
!grep ATOM 3ERK.pdb > rec.pdb
!grep SB4 3ERK.pdb > lig.pdb

# Redock the cognate ligand. Set random seed for reproducibility.
#Affinity is the Vina score of the pose. CNN pose score is a probability calculated by a convolutional neural network that the pose is a low RMSD pose. CNN affinity is the CNN's prediction of the pK (higher is better, e.g. 9 is nanomolar, 6 is micromolar).
#This particular ligand has an experimental affinity of 18 μ M so the prediction is pretty close.
!./gnina -r rec.pdb -l lig.pdb --autobox_ligand lig.pdb -o docked.sdf --seed 0

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
#@title py3Dmol

!pip install py3Dmol

import py3Dmol
v = py3Dmol.view()
v.addModel(open('rec.pdb').read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open('lig.pdb').read())
v.setStyle({'model':1},{'stick':{'colorscheme':'dimgrayCarbon','radius':.125}})
v.addModelsAsFrames(open('docked.sdf').read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.zoomTo({'model':1})
v.rotate(90)
v.animate({'interval':1000})

In [ ]:
#@title openbabel: get top ranked pose in \<#A from the crystal pose

!apt install openbabel

!obrms -f lig.pdb  docked.sdf